This is our model with the highest accuracy. 
It implements the Xception architecture, uses Image Data Augmentation and a classifier with one dense layer with l2 regularization and dropout.

In [ ]:
#Imports
import json 
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from datetime import datetime
from os import listdir
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Set seed to ensure experiment replication
SEED = 1234
tf.random.set_seed(SEED)

# Get current working directory
cwd = os.getcwd()


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#Connect to Drive to fetch data
from google.colab import drive
drive.mount('/content/drive') 


In [ ]:
#unzip the data
!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'

In [ ]:
#Create Data Generators
#Training data augmentation to training data Toggle

apply_data_augmentation = True

#Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(width_shift_range=0.10,
                                        height_shift_range=0.15,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        validation_split=0.2,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)


dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir= os.path.join(dataset_dir,"training")

#Batch size
BS = 17

#Image shape
IMG_H = 407
IMG_W = 407

num_classes=3


with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic=json.load(f)
  dataframe= pd.DataFrame(dic.items())
  dataframe.rename(columns={0:'filename', 1:'class'}, inplace=True )
  dataframe=dataframe.sample(frac=1, random_state=SEED)
  dataframe["class"]=dataframe["class"].astype('string')
  train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='training',
                                                 image_size=(407,407))
  
  validation_gen = train_data_gen.flow_from_dataframe(dataframe,
                                                 training_dir,
                                                 batch_size=BS,
                                                 target_size=(IMG_H, IMG_W),
                                                 class_mode='categorical',
                                                 shuffle= True,
                                                 seed=SEED,
                                                 subset='validation',
                                                image_size=(407,407))
  




Found 4492 validated image filenames belonging to 3 classes.
Found 1122 validated image filenames belonging to 3 classes.


In [ ]:
#create Dataset Object

#Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

#Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: validation_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()


In [ ]:
#Xception model
xception=tf.keras.applications.Xception(
      include_top=False,
      weights='imagenet',
      input_tensor=None,
      input_shape=(IMG_H, IMG_W, 3),
      pooling=max,
      classes=None,
      classifier_activation="softmax"
)
xception.summary()

83689472/83683744 [==============================] - 4s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 407, 407, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 203, 203, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 203, 203, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 203, 203, 32) 0           block1_conv1_bn[0][0]            
__________________________

In [ ]:
# Architecture: Xception -> Classifier

#Finetuning
finetuning = True

if finetuning:
    freeze_until = 21 # layer from which we want to fine-tune
    
    for layer in xception.layers[:freeze_until]:
        layer.trainable = False
else:
    xception.trainable = False

#Create model
model = tf.keras.Sequential() #we implement this in a sequential way 
model.add(xception)

# Classifier
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.6, seed=SEED))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))



In [ ]:
#Visualize created model as a table
model.summary()

#Visualize initialized weights
#model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 13, 13, 2048)      20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               524544    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 21,386,795
Trainable params: 21,175,243
Non-trainable params: 211,552
______________________________________

In [ ]:
# Optimization params

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#create Callbacks

cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/Keras/', 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'First'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)


# Early Stopping
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=9,restore_best_weights = True )
    callbacks.append(es_callback)

In [ ]:
#Model fit
model.fit(x=train_dataset,
          epochs=40,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(validation_gen),
          callbacks=callbacks)

Epoch 1/40
265/265 [==============================] - 274s 1s/step - loss: 0.9194 - accuracy: 0.7823 - val_loss: 0.6358 - val_accuracy: 0.9002
Epoch 2/40
265/265 [==============================] - 273s 1s/step - loss: 0.6111 - accuracy: 0.9114 - val_loss: 0.6265 - val_accuracy: 0.8930
Epoch 3/40
265/265 [==============================] - 273s 1s/step - loss: 0.4851 - accuracy: 0.9459 - val_loss: 0.5010 - val_accuracy: 0.9305
Epoch 4/40
265/265 [==============================] - 274s 1s/step - loss: 0.4223 - accuracy: 0.9541 - val_loss: 0.4901 - val_accuracy: 0.9251
Epoch 5/40
265/265 [==============================] - 274s 1s/step - loss: 0.3721 - accuracy: 0.9648 - val_loss: 0.4821 - val_accuracy: 0.9242
Epoch 6/40
265/265 [==============================] - 274s 1s/step - loss: 0.3268 - accuracy: 0.9726 - val_loss: 0.4630 - val_accuracy: 0.9421
Epoch 7/40
265/265 [==============================] - 274s 1s/step - loss: 0.2935 - accuracy: 0.9766 - val_loss: 0.4081 - val_accuracy: 0.9349

In [ ]:
#Make prediction on test data and export it as csv file

#Create_csv function definition
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

#import test data
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

#create predictions
predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

In [ ]:
create_csv(results, '/content/drive/My Drive/')
